In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 11.1 MB/s eta 0:00:00
Processing /colabtools/dist/google_colab-1.0.0.tar.gz (from -r requirements.txt (line 183))
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/colabtools/dist/google_colab-1.0.0.tar.gz'



In [1]:
!pip install langchain langchain-community langgraph transformers torch sentence-transformers pypdf pymupdf python-docx
!pip install faiss-gpu-cu12
!pip install langchain_ollama
!pip install langchain-groq

from langgraph.graph import StateGraph, START, END

from google.colab import userdata
import numpy as np
import pandas as pd

  Using cached langchain_ollama-0.3.8-py3-none-any.whl.metadata (2.1 kB)
  Using cached ollama-0.5.4-py3-none-any.whl.metadata (4.3 kB)
Using cached langchain_ollama-0.3.8-py3-none-any.whl (25 kB)
Using cached ollama-0.5.4-py3-none-any.whl (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 2.7 MB/s eta 0:00:00


In [2]:
from pathlib import Path
import os
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader
from typing import List

class DocumentLoader:
  def __init__(self):
    self.supported_extensions = {'.pdf', '.txt'}


  def _valid_file(self, loaded_file):
    if loaded_file in self.supported_extensions:
      return True
    return False

  def _load_pdf(self, file_path):
    loader = PyMuPDFLoader(file_path)
    return loader.load()

  def _load_text(self, file_path):
    loader = TextLoader(file_path)
    return loader.load()


  def load_document(self, file_path):
    """
    Loads documents and stores them as lists
    """
    documents = []
    # print(f"Loading documents from DL class")
    if self._valid_file(Path(file_path).suffix):
      if Path(file_path).suffix == '.pdf':
        documents.extend(self._load_pdf(file_path))
      else: documents.extend(self._load_text(file_path))
    # print(f"Loaded Documents: {documents}")
    return documents


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

class TextChunker:
  def __init__(self, chunk_size=100, chunk_overlap=0):
    self.chunk_size = chunk_size
    self.chunk_overlap = chunk_overlap

    self.recursive_splitter =  RecursiveCharacterTextSplitter(chunk_size = self.chunk_size, chunk_overlap= self.chunk_overlap, separators=["\n\n", "\n", "."])

  def chunk_documents(self, documents, strategy = 'rec'):
    if strategy=='rec':
      return self.recursive_splitter.split_documents(documents)




In [4]:
# file_path = '/content/DeepLearningBook.pdf'
file_path = '/content/speech.txt'

In [5]:
doc_loader = DocumentLoader()
text_chunker = TextChunker(500,50)
print(text_chunker.chunk_documents(doc_loader.load_document(file_path))[0])

page_content='I have called the Congress into extraordinary session because there are serious, very serious, choices of policy to be made, and made immediately, which it was neither right nor constitutionally permissible that I should assume the responsibility of making' metadata={'source': '/content/speech.txt'}


In [17]:
from sentence_transformers import SentenceTransformer


class EmbeddingModel:
  def __init__(self, model_name = 'all-mpnet-base-v2'):
    self.model_name = model_name
    self.model = SentenceTransformer(self.model_name)

  def embed_docs(self, documents):
    print(f"Embedding documents...")
    texts = [doc.page_content for doc in documents]
    embeddings = self.model.encode(texts, convert_to_tensor=False).tolist()
    return embeddings

  def embed_query(self, query):
    print(f"In embed query with query: {query}")
    return self.model.encode(query,convert_to_tensor=False).tolist()

  def get_model(self):
    return self.model



# model = SentenceTransformer("")
# embeddings =

In [7]:
import os
import numpy as np
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.schema import Document
import faiss

class VectorStore:
    def __init__(self, embedding_model):
        self.embedding_model = embedding_model
        self.faiss_index = None
        self.docstore = InMemoryDocstore()
        self.vectorstore = None
        self.documents = []

    def _embedding_function(self, x):
        if isinstance(x, str):
          return self.embedding_model.embed_query(x)
        return self.embedding_model.embed_docs


    def add_documents(self, embeddings, documents):
        print("In add documnets, class Vector Store")
        embeddings = np.array(embeddings).astype('float32')
        dim = embeddings.shape[1]
        # Create or append FAISS index
        if self.faiss_index is None:
            self.faiss_index = faiss.IndexFlatL2(dim)
        self.faiss_index.add(embeddings)
        self.documents.extend(documents)
        for i, doc in enumerate(documents):
            # print("In enumerate docs")
            # Corrected line: Passing a dictionary {doc_id: doc}
            self.docstore.add({str(len(self.documents)-len(documents)+i): doc})
        # Wrap in LangChain's FAISS VectorStore
        self.vectorstore = FAISS(
            embedding_function=self._embedding_function,  # expects a string, returns a list[float]
            index=self.faiss_index,
            docstore=self.docstore,
            index_to_docstore_id={i: str(i) for i in range(len(self.documents))}
        )
        print(self.vectorstore)


    def similarity_search(self, query: str, k: int = 5):
        print("In the vector store, similarity search.")
        if not self.vectorstore:
            raise ValueError("No documents indexed. Add documents first.")
        return self.vectorstore.similarity_search(query, k=k)

    def save_local(self, file_path: str):
        if not self.vectorstore:
            raise ValueError("No vectorstore to save.")
        self.vectorstore.save_local(file_path)

    def load_local(self, file_path: str):
        # Loading also needs the embedding function
        self.vectorstore = FAISS.load_local(
            file_path,
            self.embedding_model.embed_docs,
            allow_dangerous_deserialization=True
        )

In [93]:
# doc_loader = DocumentLoader()
# text_chunker = TextChunker(500,50)
# embedding_model = EmbeddingModel(model_name='all-mpnet-base-v2')
# vector_store = VectorStore(embedding_model)
# vector_store.add_documents(text_chunker.chunk_documents(doc_loader.load_document(file_path)))

# vector_store.save_local('/content/vector_store')
# vector_store.similarity_search("Who is the speech addressed to?")
# vector_store.similarity_search("Main points of the speech?")

In [71]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
import os
from google.colab import userdata

class DocumentSummarizer:
    def __init__(self, vector_store, model_name="llama-3.1-8b-instant"):
        self.model_name = model_name
        self.vector_store = vector_store
        os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API")
        self.chat_groq = ChatGroq(
            model=self.model_name,
            temperature=0.5
        )

    def summarize(self, query: str, top_k: int = 5):
        print("In summarize..")
        results = self.vector_store.similarity_search(query, k=top_k)
        context = "".join([doc for doc in results[0].page_content])
        # print(context)
        prompt = ChatPromptTemplate.from_messages([
            ("system", "You are an expert in summarizing documents."),
            ("user", "From the following text:\n{context}\n\n"
                     "Provide a meaningful and insightful summary. "
                     "DO NOT make up stuff. If the text is empty, say "
                     "'Information not found in documents'.")
        ])

        chain = prompt | self.chat_groq
        response = chain.invoke({"context": context})
        # print("response: ", response)
        return response.content

    def make_notes(self, query: str = None, top_k: int = 10):
        if query:
            results = self.vector_store.similarity_search(query, k=top_k)
        else:
            # if no query, just take a representative sample
            results = self.vector_store.similarity_search("", k=top_k)

        context = "".join([doc for doc in results[0].page_content])

        prompt = ChatPromptTemplate.from_messages([
            ("system",
             "You are an expert in analysing documents and creating meaningful notes."),
            ("user",
             "Based on the following text:\n{context}\n\n"
             "Create structured notes with:\n"
             "1. Key Points: [main ideas]\n"
             "2. Important Details: [supporting information]\n"
             "3. Actionable Insights: [what can be done]")
        ])
        chain = prompt | self.chat_groq
        response = chain.invoke({"context": context})
        return response.content

    def create_flashcards(self, query:str):
        if not query:
              raise ValueError("Need to provide query!!")

        prompt = ChatPromptTemplate.from_messages([
            ("system",
             "You are an expert in creating flashcards based on the provided information."),
            ("user",
             "Based on the following text:\n{query}\n\n"
             "Create a flash card with:\n"
             "1. The flash card should contain a short (2 to 3 lines), memorable answer that captures the key point(s)]\n"
             "2. Keep the language simple and focused so it’s easy to memorize.\n"
             "3. If the concept is complex, break it down into a core definition, formula, or key takeaway.")
        ])

        chain = prompt | self.chat_groq
        response = chain.invoke({"query": query})
        return response.content



In [95]:
# ds = DocumentSummarizer(vector_store)
# print(ds.summarize("what are Recurrent Neural Networks?"))
# print(ds.make_notes("Recurrent Neural Networks"))
# print(ds.create_flashcards(query="""A
# Gaussian Process with a given kernel defines a prior over functions."""))
# query="""Now suppose that we wish to employ the strategy mentioned above, where we condition
# only on the 𝜏previous time steps, i.e., 𝑥𝑡−1,...,𝑥𝑡−𝜏, rather than the entire sequence history
# 𝑥𝑡−1,...,𝑥1. Whenever we can throw away the history beyond the previous 𝜏steps without
# any loss in predictive power, we say that the sequence satisfies a Markov condition, i.e., that
# the future is conditionally independent of the past, given the recent history. When 𝜏 = 1,
# we say that the data is characterized by a first-order Markov model, and when 𝜏 = 𝑘, we
# say that the data is characterized by a 𝑘th-order Markov model."""

# query="""The new policy has swept every restriction aside. Vessels of every kind, whatever their flag, their character, their cargo, their destination, their errand, have been ruthlessly sent to the bottom without warning and without thought of help or mercy for those on board, the vessels of friendly neutrals along with those of belligerents. Even hospital ships and ships carrying relief to the sorely bereaved and stricken people of Belgium, though the latter were provided with safe conduct through the proscribed areas by the German government itself and were distinguished by unmistakable marks of identity, have been sunk with the same reckless lack of compassion or of principle.

# I was for a little while unable to believe that such things would in fact be done by any government that had hitherto subscribed to the humane practices of civilized nations. International law had its origin in the attempt to set up some law which would be respected and observed upon the seas, where no nation had right of dominion and where lay the free highways of the world. By painful stage after stage has that law been built up, with meager enough results, indeed, after all was accomplished that could be accomplished, but always with a clear view, at least, of what the heart and conscience of mankind demanded."""

# print(ds.make_notes())

In [19]:
from typing import TypedDict, List, Optional, Annotated

class QuantumNotesState(TypedDict):
  file_path: str
  task:str
  query:Optional[str]
  documents: Optional[List[str]]
  summary: Optional[str]
  notes: Optional[str]
  flashcards: Optional[str]
  errors: Optional[List[Exception]]
  chunks: Optional[List[str]]
  embeddings: Optional[List[float]]
  vector_store: Optional[str]
  context: Optional[str]
  model_name: Optional[str]



In [32]:
class DocumentProcessingAgent:
  def __init__(self):
    self.doc_loader = DocumentLoader()
    self.text_chunker = TextChunker()
    self.embedding_model = EmbeddingModel()
    # self.vector_store = VectorStore()


  def load_documents_node(self, state:QuantumNotesState):
    print("In load_docs_node...")
    file_path = state.get('file_path', "")
    if file_path:
      print(f"Loading documents from {file_path}")
      documents = self.doc_loader.load_document(file_path)
      return {"documents": documents}
    else:
      return {"documents": []}

  def text_chunker_node(self, state:QuantumNotesState):
    print("In chunking")
    documents = state.get('documents', [])
    # chunk_size = state.get('chunk_size', 500)
    # chunk_overlap = state.get('chunk_overlap', 50)
    if documents:
      chunks =  self.text_chunker.chunk_documents(documents)
      return {"chunks": chunks}
    return {"chunks": []}

  def embeddings_and_vectorstore_node(self, state:QuantumNotesState):
    print("In embeddings and vectorstore")
    chunks = state.get("chunks", [])
    vector_store = VectorStore(self.embedding_model)
    if chunks:
      print("Adding documents to vector store...")
      embeddings = self.embedding_model.embed_docs(chunks)
      print("After Embeddings..")
      vector_store.add_documents(embeddings, chunks)
      return {"embeddings": embeddings, "vector_store": vector_store}

    return {"embeddings": [], "vector_store":[]}





In [56]:
class SummarizingAgent:
  def __init__(self, model_name:str="llama-3.1-8b-instant"):
    self.model_name = model_name
  def make_summary_node(self, state:QuantumNotesState):
    print("In make_summary")
    vector_store = state.get("vector_store")

    ds = DocumentSummarizer(vector_store, self.model_name)
    query = state.get("query", "")
    # query = state.get("summary_query", "")
    summary = ds.summarize(query)
    print("Printing summary....")
    return {"summary": summary}


In [57]:
class NoteMakingAgent:
  def __init__(self, model_name:str="llama-3.1-8b-instant"):
    self.model_name = model_name

  def make_notes_node(self, state:QuantumNotesState):
    print("In make_notes")
    vector_store = state.get("vector_store")
    ds = DocumentSummarizer(vector_store, self.model_name)
    query = state.get("query", "")
    # query = state.get("notes_query", "")
    notes = ds.make_notes(query)
    return {"notes": notes}


In [58]:
class FlashCardMakingAgent:
  def __init__(self, model_name:str="llama-3.1-8b-instant"):
    self.model_name = model_name

  def make_flashcards_node(self, state:QuantumNotesState):
    print("In make_flashcards")
    vector_store = state.get("vector_store")
    ds = DocumentSummarizer(vector_store, self.model_name)
    query = state.get("query", "")
    # query = state.get("flash_query", "")
    flashcard = ds.create_flashcards(query)
    return {"flashcards": flashcard}

In [59]:

class QuantumNotesAgent:
  def __init__(self):
    self.dpa = DocumentProcessingAgent()
    self.sa = SummarizingAgent()
    self.nma = NoteMakingAgent()
    self.fca = FlashCardMakingAgent()
    self.graph = None

  def _check_task(self, state:QuantumNotesState):
    if state.get("task") == "summarize":
      return "make_summary"
    elif state.get("task") == "make_notes":
      return "make_notes"
    elif state.get("task") == "make_flashcards":
      return "make_flashcards"
    else:
      return END # or handle unsupported task

  def _build_graph(self):
    print(f"Builder...")
    builder = StateGraph(QuantumNotesState)
    builder.add_node("load_documents", self.dpa.load_documents_node)
    builder.add_node("text_chunker", self.dpa.text_chunker_node)
    builder.add_node("embeddings_and_vectorstore", self.dpa.embeddings_and_vectorstore_node)
    builder.add_node("make_summary", self.sa.make_summary_node)
    builder.add_node("make_notes", self.nma.make_notes_node)
    builder.add_node("make_flashcards", self.fca.make_flashcards_node)


    builder.add_edge(START, "load_documents")
    builder.add_edge("load_documents", "text_chunker")
    builder.add_edge("text_chunker", "embeddings_and_vectorstore")
    builder.add_conditional_edges("embeddings_and_vectorstore", self._check_task, ["make_summary", "make_notes", "make_flashcards"])
    builder.add_edge("make_summary", END)
    builder.add_edge("make_notes", END)
    builder.add_edge("make_flashcards", END)
    self.graph = builder.compile()

  def process_docs(self, state:QuantumNotesState):
    if not self.graph:
      self._build_graph()
    return self.graph.invoke(state)

In [68]:
initial_state = {
    "file_path": "/content/speech.txt",
    "query": "what is the speech about?",
    "task": "summarize"
}

qa = QuantumNotesAgent()
result = qa.process_docs(initial_state)
print(result['summary'])

Builder...
In load_docs_node...
Loading documents from /content/speech.txt
In chunking
In embeddings and vectorstore
Adding documents to vector store...
Embedding documents...


After Embeddings..
In add documnets, class Vector Store
In make_summary
In summarize..
In the vector store, similarity search.
In embed query with query: what is the speech about?
. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free
Printing summary....
The text emphasizes the importance of fighting for fundamental rights and freedoms, including democracy, individual voice in governance, and the rights of small nations. It also highlights the goal of achieving a universal dominion of right through international cooperation among free peoples, with the ultimate aim of bringing peace, safety, and freedom to all nati

In [72]:
initial_state = {
    "file_path": "/content/speech.txt",
    "query": "The text emphasizes the importance of fighting for fundamental rights and freedoms, including democracy, individual voice in governance, and the rights of small nations. It also highlights the goal of achieving a universal dominion of right through international cooperation among free peoples, with the ultimate aim of bringing peace, safety, and freedom to all nations.",
    "task": "make_notes"
}

qa = QuantumNotesAgent()
result = qa.process_docs(initial_state)


Builder...
In load_docs_node...
Loading documents from /content/speech.txt
In chunking
In embeddings and vectorstore
Adding documents to vector store...
Embedding documents...


After Embeddings..
In add documnets, class Vector Store
In make_notes
In the vector store, similarity search.
In embed query with query: The text emphasizes the importance of fighting for fundamental rights and freedoms, including democracy, individual voice in governance, and the rights of small nations. It also highlights the goal of achieving a universal dominion of right through international cooperation among free peoples, with the ultimate aim of bringing peace, safety, and freedom to all nations.
<built-in method keys of dict object at 0x791941f06fc0>


In [73]:
print(result.keys())

dict_keys(['file_path', 'task', 'query', 'documents', 'notes', 'chunks', 'embeddings', 'vector_store'])


In [75]:
print(result['notes'])

**Structured Notes**

**Document:** Speech or passage about the importance of democracy and human rights

**Key Points:**

1. The right is more precious than peace.
2. The importance of democracy and human rights.
3. The need for universal dominion of right through a concert of free peoples.

**Important Details:**

1. The right to have a voice in one's own government.
2. The rights and liberties of small nations.
3. The goal of bringing peace and safety to all nations.
4. The ultimate goal of making the world free.

**Actionable Insights:**

1. **Advocate for democracy:** Support and promote democratic institutions and practices in your community and country.
2. **Empower marginalized voices:** Amplify the voices of those who are oppressed or marginalized, and work towards creating a more inclusive and equitable society.
3. **Promote international cooperation:** Support international organizations and initiatives that promote peace, safety, and human rights, and work towards creating 

In [63]:
result['task']

'summarize'

In [64]:
result['query']

'what is the speech about?'

In [69]:
result['summary']

'The text emphasizes the importance of fighting for fundamental rights and freedoms, including democracy, individual voice in governance, and the rights of small nations. It also highlights the goal of achieving a universal dominion of right through international cooperation among free peoples, with the ultimate aim of bringing peace, safety, and freedom to all nations.'

In [76]:
initial_state = {
    "file_path": "/content/speech.txt",
    "task": "make_notes"
}

qa = QuantumNotesAgent()
result = qa.process_docs(initial_state)
print(result['notes'])

Builder...
In load_docs_node...
Loading documents from /content/speech.txt
In chunking
In embeddings and vectorstore
Adding documents to vector store...
Embedding documents...


After Embeddings..
In add documnets, class Vector Store
In make_notes
In the vector store, similarity search.
In embed query with query: 
However, it seems like there is no provided text for me to create notes from. Please provide the text you would like me to analyze and create structured notes from.


In [77]:
initial_state = {
    "file_path": "/content/speech.txt",
    "query": "Sovereignity means supreme power especially over a body politic.",
    "task": "make_flashcards"
}

qa = QuantumNotesAgent()
result = qa.process_docs(initial_state)
print(result['notes'])

Builder...
In load_docs_node...
Loading documents from /content/speech.txt
In chunking
In embeddings and vectorstore
Adding documents to vector store...
Embedding documents...


After Embeddings..
In add documnets, class Vector Store
In make_flashcards


KeyError: 'notes'

In [78]:
print(result.keys())

dict_keys(['file_path', 'task', 'query', 'documents', 'flashcards', 'chunks', 'embeddings', 'vector_store'])


In [79]:
print(result['flashcards'])

**Flashcard:**

**Front:** Sovereignty
**Back:** Supreme power over a country or government.
